https://jira.x5.ru/browse/CVMXC-2679

In [1]:
name = 'CVMXC-2679_kotex'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *
from logic_filters import *
from segmentation import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/14 14:24:47 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Выбираем гостей нужного юзкейса

In [7]:
usecase = ['churn', 'frequency']
dt = datetime.date(2022, 2, 11)

In [8]:
customers_usecase_churn = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase[0]))
#                      .filter(F.col('lifetime') >= lifetime)
#                      .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )
customers_usecase_freq = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase[1]))
#                      .filter(F.col('lifetime') >= lifetime)
#                      .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )

In [9]:
print(customers_usecase_churn.count()) #16775430
print(customers_usecase_freq.count()) #16952965

16775430


[Stage 2:====================================================>     (9 + 1) / 10]

16952965


### 1 проверяем на доступность отобранных гостей в определенную дату

In [8]:
check_date = '2022-03-07'

In [9]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_churn, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .withColumn('usecase', F.lit('churn'))
                                    .select('customer_rk', 'usecase')
                                    .distinct()
           )

 Время выполнения: 0:00:20


In [10]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_freq, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .withColumn('usecase', F.lit('freq'))
                                    .select('customer_rk', 'usecase')
                                    .distinct()
           )

 Время выполнения: 0:00:11


In [11]:
seg_sms = seg_sms1.union(seg_sms2).distinct()

In [12]:
seg_sms.write.parquet('temp2679_new1', mode='overwrite')
seg_sms = spark.read.parquet('temp2679_new1')

[Stage 4:=====================================================> (161 + 5) / 166]22/02/11 15:58:35 ERROR cluster.YarnScheduler: Lost executor 34 on mn-hd0236.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.


In [13]:
seg_sms.count() #

15055810

### 2 проверяем на доступность отобранных гостей в определенную дату

In [14]:
slip_check1 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_churn, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .withColumn('usecase', F.lit('churn'))
                                    .select('customer_rk', 'usecase')
                                    .distinct()
           )

 Время выполнения: 0:00:01


In [15]:
slip_check2 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_freq, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .withColumn('usecase', F.lit('freq'))
                                    .select('customer_rk', 'usecase')
                                    .distinct()
           )

 Время выполнения: 0:00:01


In [16]:
slip_check = slip_check1.union(slip_check2).distinct()

In [17]:
slip_check.write.parquet('temp2679_new2', mode='overwrite')
slip_check = spark.read.parquet('temp2679_new2')

22/02/11 16:08:39 ERROR client.TransportClient: Failed to send RPC RPC 7987644826857027621 to /192.168.234.57:57280: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)


In [18]:
slip_check.count() #

29218454

In [19]:
seg_sms.join(slip_check, 'customer_rk', 'inner').count()

15055810

### Проверка на доступность плю в магазинах

РФ

In [20]:
plu_codes = [6075, 3364392, 3635965, 3364389, 3647604, 6076, 3017398]

In [21]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            .selectExpr('store_id as plant')
            .distinct()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

22011

### Heavy users

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [22]:
start_date = datetime.date(2021, 9, 8)
end_date = datetime.date(2022, 2, 7)

In [23]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [24]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [25]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(F.col('loyalty_cardholder_age_yrs') < 40)
                        .filter(F.col('gender_dk') == 'F')
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(slip_check, on='customer_rk', how='inner')

In [26]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [27]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [28]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [29]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
# syntethic_category_id = [96,57,75,73,101,55,56,22]
plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [30]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [31]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [32]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [33]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [34]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [35]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 1)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [38]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group', 'usecase')
                  .distinct()
      )

In [39]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group', 'usecase')
                  .distinct()
      )

In [40]:
seg = seg1.union(seg2)

In [10]:
# seg.write.parquet('temp2679_seg_new1', mode='overwrite')
seg = spark.read.parquet('temp2679_seg_new1')

In [42]:
seg_pd1 = seg.toPandas()

In [44]:
seg_pd1.head()

,customer_rk,group,usecase
0,59046148,2,churn
1,129306239,2,freq
2,38972934,2,freq
3,94737949,2,freq
4,75138671,2,freq


### Конкуренты

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [45]:
start_date = datetime.date(2021, 9, 8)
end_date = datetime.date(2022, 2, 7)

In [46]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [47]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [48]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(F.col('loyalty_cardholder_age_yrs') < 40)
                        .filter(F.col('gender_dk') == 'F')
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(slip_check, on='customer_rk', how='inner')

In [49]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [50]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [51]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [52]:
plu_hierarchy_lvl_4_dk = ['FD0911001']
# syntethic_category_id = [96,57,75,73,101,55,56,22]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [53]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [54]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [55]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [56]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [57]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [58]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 1)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [59]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group', 'usecase')
                  .distinct()
      )

In [60]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group', 'usecase')
                  .distinct()
      )

In [61]:
seg_2 = seg1.union(seg2)

In [62]:
# seg_2.write.parquet('temp2679_seg_new2', mode='overwrite')
seg_2 = spark.read.parquet('temp2679_seg_new2')

In [63]:
seg_pd2 = seg_2.toPandas()

### Остальные

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [64]:
start_date = datetime.date(2021, 9, 8)
end_date = datetime.date(2022, 2, 7)

In [65]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [66]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [67]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(F.col('loyalty_cardholder_age_yrs') < 40)
                        .filter(F.col('gender_dk') == 'F')
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(slip_check, on='customer_rk', how='inner')

In [68]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [69]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [70]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [71]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [72]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [73]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group', 'usecase')
                  .distinct()
      )

In [74]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group', 'usecase')
                  .distinct()
      )

In [75]:
seg_3 = seg1.union(seg2)

In [76]:
seg_3.write.parquet('temp2679_seg_new3', mode='overwrite')
seg_3 = spark.read.parquet('temp2679_seg_new3')

In [77]:
seg_3.count()

3767170

In [78]:
seg_pd3 = seg_3.toPandas()

### Все юзкейсы

### Выбираем гостей нужного юзкейса

In [79]:
usecase = ['churn', 'frequency', 'cross', 'upgrade', 'ump']
dt = datetime.date.today()

In [80]:
customers_usecase_churn = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase[0]))
#                      .filter(F.col('lifetime') >= lifetime)
#                      .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )
customers_usecase_freq = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase[1]))
#                      .filter(F.col('lifetime') >= lifetime)
#                      .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )
customers_usecase_all = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
#                      .filter(F.col('lifetime') >= lifetime)
#                      .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )

In [81]:
print(customers_usecase_all.count()) #15 861 048

[Stage 178:====================================================>(259 + 1) / 260]

49577608


### 1 проверяем на доступность отобранных гостей в определенную дату

In [82]:
check_date = '2022-03-07'

In [83]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_all, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [84]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_all, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:02


In [85]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_all, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:06


In [86]:
seg_sms4 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_all, 
                                         usecase_name=usecase[3], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [87]:
seg_sms5 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_all, 
                                         usecase_name=usecase[4], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:02


In [88]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).union(seg_sms4).union(seg_sms5).distinct()

In [90]:
# seg_sms.write.parquet('temp2679_new3', mode='overwrite')
seg_sms = spark.read.parquet('temp2679_new3')

In [90]:
seg_sms.count() #

20314044

### 2 проверяем на доступность отобранных гостей в определенную дату

In [91]:
slip_check1 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_all, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:01


In [92]:
slip_check2 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_all, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:01


In [93]:
slip_check3 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_all, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:01


In [94]:
slip_check4 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_all, 
                                         usecase_name=usecase[3], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:01


In [95]:
slip_check5 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase_all, 
                                         usecase_name=usecase[4], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:01


In [96]:
slip_check = slip_check1.union(slip_check2).union(slip_check3).union(slip_check4).union(slip_check5).distinct()

In [91]:
# slip_check.write.parquet('temp2679_new4', mode='overwrite')
slip_check = spark.read.parquet('temp2679_new4')

In [98]:
slip_check.count() #

41150354

In [99]:
seg_sms.join(slip_check, 'customer_rk', 'inner').count()

20314044

### Проверка на доступность плю в магазинах

РФ

In [92]:
plu_codes = [6075, 3364392, 3635965, 3364389, 3647604, 6076, 3017398]

In [93]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            .selectExpr('store_id as plant')
            .distinct()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

22017

### Heavy users

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [102]:
start_date = datetime.date(2021, 9, 8)
end_date = datetime.date(2022, 2, 7)

In [103]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [104]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [105]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(F.col('gender_dk') == 'F')
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(slip_check, on='customer_rk', how='inner')

In [106]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [107]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

22/02/11 19:24:25 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.57:47512 is closed


In [108]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [109]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
# syntethic_category_id = [96,57,75,73,101,55,56,22]
plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [110]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [111]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [112]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [113]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [114]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [115]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 1)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [116]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [117]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [118]:
seg_4 = seg1.union(seg2)

In [11]:
# seg_4.write.parquet('temp2679_seg_new4', mode='overwrite')
seg_4 = spark.read.parquet('temp2679_seg_new4')

In [120]:
seg_pd4 = seg_4.toPandas()

In [121]:
seg_pd4.head()

,customer_rk,group
0,17553201,2
1,20146819,2
2,28177697,2
3,33842372,2
4,30331254,2


### Конкуренты

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [122]:
start_date = datetime.date(2021, 9, 8)
end_date = datetime.date(2022, 2, 7)

In [123]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [124]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [125]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(F.col('gender_dk') == 'F')
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(slip_check, on='customer_rk', how='inner')

In [126]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [127]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [128]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [129]:
plu_hierarchy_lvl_4_dk = ['FD0911001']
# syntethic_category_id = [96,57,75,73,101,55,56,22]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [130]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [131]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [132]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [133]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [134]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [135]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 1)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [138]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [139]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [140]:
seg_5 = seg1.union(seg2)

In [12]:
# seg_5.write.parquet('temp2679_seg_new5', mode='overwrite')
seg_5 = spark.read.parquet('temp2679_seg_new5')

In [142]:
seg_pd5 = seg_5.toPandas()

### Без пола и без возраста

### Heavy users

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [94]:
start_date = datetime.date(2021, 9, 8)
end_date = datetime.date(2022, 2, 7)

In [95]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [96]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [97]:
names = ['''Ава, Авгуполинста, Августина, Авдотья, Аврора, Агапия, Агата, Агафия, Агафья, Аглая, Агния, Агриппина, Ада, Аделаида, 
Аделина, Аделия, Адель, Адриана, Азалия, Аида, Аксинья, Алана, Алевтина, Алекса, Александра, Александрина, Алёна, 
Алеся, Алина, Алиса, Алла, Альба, Альбина, Аля, Амалия, Амелия, Амина, Анастасия, Ангелина, Анжела, Анжелика, 
Анисья, Анна, Антонина, Анфиса, Аполлинария, Арабелла, Ариана, Арина, Арсена, Ассоль, Ася, Аэлита, Аюта, Бажена, 
Беатрис, Бела, Белла, Берта, Богдана, Божена, Бриллиант, Бронислава, Бьянка, Бэлла, Валентина, Валерия, Ванда, 
Варвара, Василина, Василиса, Васса, Венера, Вера, Вероника, Вета, Викторина, Виктория, Вилена, Виола, Виолетта, 
Вит, Вита, Виталина, Виталия, Влада, Владана, Владислава, Владлена, Волга, Габриэлла, Галина, Гвоздика, Георгина, 
Гертруда, Глафира, Грета, Даздраперма, Даниэла, Дания, Дара, Дарина, Дария, Декабрина, Дарья, Дженна, Диамара, 
Диана, Дина, Долорес, Доминика, Домна, Домника, Дора, Ева, Евангелина, Ева, Евгения, Евдокия, Екатерина, Елен, 
Алёна, Елена, Елизавета, Есения, Ефросинья, Жаклин, Жанна, Жозефина, Жоржина, Залина, Зара, Зинаида, Зиновия, 
Зита, Злата, Златослава, Зореслава, Зоя, Иванна, Ивета, Иветта, Ида, Изабелла, Иллирика, Илона, Инесса, Инна, 
Интерна, Иоанна, Иона, Ира, Ираида, Ирина, Ирма, Искра, Исталина, Иулиания, Июлия, Ия, Калерия, Камилла, 
Капитолина, Кара, Карина, Кармия, Каролина, Катерина, Кира, Клавдия, Клара, Клёна, Клементина, Кора, Корнелия, 
Кристина, Ксения, Лада, Лана, Лара, Лариса, Лаура, Ленина, Леокадия, Леона, Лера, Леся, Лета, Лиана, Лидия, 
Лика, Лили, Лилиана, Лилия, Лина, Лия, Лола, Лолита, Лора, Луиза, Лукерья, Лукина, Лукия, Любава, Любовь, 
Людмила, Люсиль, Люсьена, Люция, Люче, Ляля, Магда, Магдалена, Мадлен, Майя, Макария, Маргарита, Мариа, 
Марианна, Марика, Марина, Мария, Марта, Марфа, Марьяна, Мелания, Мила, Милада, Милана, Милен, Милена, Милица,
Милослава, Мира, Мирослава, Мирра, Мия, Моника, Муза, Надежда, Нана, Наталия, Наталья, Нева, Нелли, Неонилла, 
Ника, Николь, Нина, Нинель, Нора, Одетта, Оксана, Октябрина, Олеся, Оливия, Олимпиада, Ольга, Павла, Павлина, 
Патриция, Паула, Пейтон, Пелагея, Платонида, Победа, Полина, Прасковья, Рада, Раиса, Рамина, Ребекка, Револа, 
Регина, Резеда, Рена, Рената, Рианна, Рикарда, Римма, Рина, Рита, Рогнеда, Роза, Роксолана, Ростислава, Русалина, 
Руслана, Руфина, Сабина, Сабрина, Саломея, Сандра, Сара, Сауле, Светлана, Святослава, Севастьяна, Сентябрина, 
Серафима, Сесилия, Сильвия, Снежана, Соломия, Соня, София, Софья, Союза, Спартакиада, Станислава, Стелла, 
Стефания, Сусанна, Таисия, Тамара, Тамила, Татьяна, Таяна, Теона, Тереза, Тина, Ульяна, Урсула, Услада, 
Устинья, Фаина, Фёкла, Флора, Флоренс, Флорентина, Флоренция, Флориана, Фотиния, Хилари, Хома, Цагана, 
Цветана, Цецилия, Челси, Шарлотта, Шейла, Эвелина, Элеонора, Элиана, Элиза, Элизар, Элина, Элла, Эльвина, 
Эльвира, Эля, Эмили, Эмилия, Эмма, Эрмина, Эсмеральда, Этель, Юлианна, Юлия, Юния, Ядвига, Яна, Янина, Ярина, 
Ярослава'''
]
names = names[0].split(',')
for i in range(len(names)):
    names[i] = names[i].replace('\n', '')
    names[i] = names[i].replace(' ', '')

In [98]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(F.col('first_nm').isin(names))
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(slip_check, on='customer_rk', how='inner')

In [99]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [100]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [101]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [102]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
# syntethic_category_id = [96,57,75,73,101,55,56,22]
plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [103]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [104]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [105]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [106]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [107]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [108]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 1)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [109]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [110]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [111]:
seg_6 = seg1.union(seg2)

In [112]:
seg_6.write.parquet('temp2679_seg_new6', mode='overwrite')
seg_6 = spark.read.parquet('temp2679_seg_new6')

In [146]:
seg_pd6 = seg_6.toPandas()

### Конкуренты

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [113]:
start_date = datetime.date(2021, 9, 8)
end_date = datetime.date(2022, 2, 7)

In [114]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [115]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [116]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(F.col('first_nm').isin(names))
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(slip_check, on='customer_rk', how='inner')

In [117]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [118]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [119]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [120]:
plu_hierarchy_lvl_4_dk = ['FD0911001']
# syntethic_category_id = [96,57,75,73,101,55,56,22]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [121]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [122]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [123]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [124]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [125]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [126]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 1)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [127]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [128]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [129]:
seg_7 = seg1.union(seg2)

In [130]:
seg_7.write.parquet('temp2679_seg_new7', mode='overwrite')
seg_7 = spark.read.parquet('temp2679_seg_new7')

In [145]:
seg_pd7 = seg_7.toPandas()

### Остальные

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [131]:
start_date = datetime.date(2021, 9, 8)
end_date = datetime.date(2022, 2, 7)

In [132]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [133]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [134]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(F.col('first_nm').isin(names))
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(slip_check, on='customer_rk', how='inner')

In [135]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [136]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [137]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [138]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [139]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [140]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [141]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [142]:
seg_8 = seg1.union(seg2)

In [143]:
seg_8.write.parquet('temp2679_seg_new8', mode='overwrite')
seg_8 = spark.read.parquet('temp2679_seg_new8')

[Stage 121:>           (10 + 190) / 200][Stage 125:>            (0 + 180) / 200]/ 200]]]]]]22/02/14 17:43:25 ERROR cluster.YarnScheduler: Lost executor 501 on mn-hdcn044.x5.ru: Container killed by YARN for exceeding memory limits.  4.5 GB of 4.5 GB physical memory used. Consider boosting spark.yarn.executor.memoryOverhead or disabling yarn.nodemanager.vmem-check-enabled because of YARN-4714.
[Stage 121:>           (10 + 190) / 200][Stage 125:>            (0 + 175) / 200]22/02/14 17:43:26 ERROR cluster.YarnScheduler: Lost executor 509 on mn-hdcn054.x5.ru: Container killed by YARN for exceeding memory limits.  4.5 GB of 4.5 GB physical memory used. Consider boosting spark.yarn.executor.memoryOverhead or disabling yarn.nodemanager.vmem-check-enabled because of YARN-4714.
[Stage 121:=>          (32 + 168) / 200][Stage 125:>            (0 + 200) / 200]22/02/14 17:43:50 ERROR cluster.YarnScheduler: Lost executor 491 on mn-hdcn038.x5.ru: Container killed by YARN for exceeding memory limits.  

In [144]:
seg_pd8 = seg_8.toPandas()

In [ ]:
seg_pd1.to_csv('seg_pd1.csv', index=False)
seg_pd2.to_csv('seg_pd2.csv', index=False)
seg_pd3.to_csv('seg_pd3.csv', index=False)
seg_pd4.to_csv('seg_pd4.csv', index=False)
seg_pd5.to_csv('seg_pd5.csv', index=False)

In [147]:
seg_pd6.to_csv('seg_pd6.csv', index=False)
seg_pd7.to_csv('seg_pd7.csv', index=False)
seg_pd8.to_csv('seg_pd8.csv', index=False)

In [17]:
seg_pd1 = pd.read_csv('seg_pd1.csv')
seg_pd2 = pd.read_csv('seg_pd2.csv')
seg_pd3 = pd.read_csv('seg_pd3.csv')

In [32]:
seg_pd_1_churn_sms = seg_pd1[(seg_pd1.usecase == 'churn') & (seg_pd1.group == 1)]['customer_rk']
seg_pd_1_churn_slip = seg_pd1[(seg_pd1.usecase == 'churn') & (seg_pd1.group == 2)]['customer_rk']
seg_pd_1_freq_sms = seg_pd1[(seg_pd1.usecase == 'freq') & (seg_pd1.group == 1)]['customer_rk']
seg_pd_1_freq_slip = seg_pd1[(seg_pd1.usecase == 'freq') & (seg_pd1.group == 2)]['customer_rk']
print('churn_sms: {}, churn_slip: {}, freq_sms: {}, freq_slip: {} (all_sms: {}, all_slip: {})'.format(len(seg_pd_1_churn_sms),
                                                                                                     len(seg_pd_1_churn_slip),
                                                                                                     len(seg_pd_1_freq_sms),
                                                                                                     len(seg_pd_1_freq_slip),
                                                                                                      len(seg_pd_1_churn_sms)+
                                                                                                      len(seg_pd_1_freq_sms),
                                                                                                      len(seg_pd_1_churn_slip)+
                                                                                                      len(seg_pd_1_freq_slip)))

churn_sms: 19010, churn_slip: 33947, freq_sms: 65527, freq_slip: 102667 (all_sms: 84537, all_slip: 136614)


In [33]:
seg_pd_2_churn_sms = list(set(seg_pd2[(seg_pd2.usecase == 'churn') & (seg_pd2.group == 1)]['customer_rk']) - set(seg_pd_1_churn_sms))
seg_pd_2_churn_slip = list(set(seg_pd2[(seg_pd2.usecase == 'churn') & (seg_pd2.group == 2)]['customer_rk']) - set(seg_pd_1_churn_slip))
seg_pd_2_freq_sms = list(set(seg_pd2[(seg_pd2.usecase == 'freq') & (seg_pd2.group == 1)]['customer_rk']) - set(seg_pd_1_freq_sms))
seg_pd_2_freq_slip = list(set(seg_pd2[(seg_pd2.usecase == 'freq') & (seg_pd2.group == 2)]['customer_rk']) - set(seg_pd_1_freq_slip))
print('churn_sms: {}, churn_slip: {}, freq_sms: {}, freq_slip: {} (all_sms: {}, all_slip: {})'.format(len(seg_pd_2_churn_sms),
                                                                                                     len(seg_pd_2_churn_slip),
                                                                                                     len(seg_pd_2_freq_sms),
                                                                                                     len(seg_pd_2_freq_slip),
                                                                                                      len(seg_pd_2_churn_sms)+
                                                                                                      len(seg_pd_2_freq_sms),
                                                                                                      len(seg_pd_2_churn_slip)+
                                                                                                      len(seg_pd_2_freq_slip)))

churn_sms: 60852, churn_slip: 109539, freq_sms: 202051, freq_slip: 319170 (all_sms: 262903, all_slip: 428709)


In [34]:
seg_pd_3_churn_sms = list(set(seg_pd3[(seg_pd3.usecase == 'churn') & (seg_pd3.group == 1)]['customer_rk']) - set(seg_pd_2_churn_sms) - set(seg_pd_1_churn_sms))
seg_pd_3_churn_slip = list(set(seg_pd3[(seg_pd3.usecase == 'churn') & (seg_pd3.group == 2)]['customer_rk']) - set(seg_pd_2_churn_slip) - set(seg_pd_1_churn_slip))
seg_pd_3_freq_sms = list(set(seg_pd3[(seg_pd3.usecase == 'freq') & (seg_pd3.group == 1)]['customer_rk']) - set(seg_pd_2_freq_sms) - set(seg_pd_1_freq_sms))
seg_pd_3_freq_slip = list(set(seg_pd3[(seg_pd3.usecase == 'freq') & (seg_pd3.group == 2)]['customer_rk']) - set(seg_pd_2_freq_slip) - set(seg_pd_1_freq_slip))
print('churn_sms: {}, churn_slip: {}, freq_sms: {}, freq_slip: {} (all_sms: {}, all_slip: {})'.format(len(seg_pd_3_churn_sms),
                                                                                                     len(seg_pd_3_churn_slip),
                                                                                                     len(seg_pd_3_freq_sms),
                                                                                                     len(seg_pd_3_freq_slip),
                                                                                                      len(seg_pd_3_churn_sms)+
                                                                                                      len(seg_pd_3_freq_sms),
                                                                                                      len(seg_pd_3_churn_slip)+
                                                                                                      len(seg_pd_3_freq_slip)))

churn_sms: 458145, churn_slip: 775936, freq_sms: 626886, freq_slip: 993440 (all_sms: 1085031, all_slip: 1769376)


In [37]:
seg_pd_4_churn = seg_4.join(customers_usecase_churn, 'customer_rk', 'inner').toPandas()
seg_pd_4_freq = seg_4.join(customers_usecase_freq, 'customer_rk', 'inner').toPandas()
seg_pd_4_others = seg_4.join(customers_usecase_churn, 'customer_rk', 'left_anti').join(customers_usecase_freq, 'customer_rk', 'left_anti').toPandas()

In [49]:
seg_pd_4_churn_sms = seg_pd_4_churn[seg_pd_4_churn.group == '1']['customer_rk']
seg_pd_4_churn_slip = seg_pd_4_churn[seg_pd_4_churn.group == '2']['customer_rk']
seg_pd_4_freq_sms = seg_pd_4_freq[seg_pd_4_freq.group == '1']['customer_rk']
seg_pd_4_freq_slip = seg_pd_4_freq[seg_pd_4_freq.group == '2']['customer_rk']
seg_pd_4_others_sms = seg_pd_4_others[seg_pd_4_others.group == '1']['customer_rk']
seg_pd_4_others_slip = seg_pd_4_others[seg_pd_4_others.group == '2']['customer_rk']

In [52]:
seg_pd_4_others_sms = seg_pd_4_others_sms.astype('str')
seg_pd_4_others_slip = seg_pd_4_others_slip.astype('str')

In [53]:
cross4_sms, upgrade4_sms, ump4_sms = get_usecases(pd.DataFrame({'customer_rk': seg_pd_4_others_sms.to_list()}))
cross4_slip, upgrade4_slip, ump4_slip = get_usecases(pd.DataFrame({'customer_rk': seg_pd_4_others_slip.to_list()}))

In [55]:
print('churn_sms: {}, churn_slip: {}, freq_sms: {}, freq_slip: {}, cross_sms: {}, cross_slip: {}, upgrade_sms: {}, upgrade_slip: {}, ump_sms: {}, ump_slip: {} (all_sms: {}, all_slip: {})'
      .format(len(seg_pd_4_churn_sms),
              len(seg_pd_4_churn_slip), 
              len(seg_pd_4_freq_sms),
             len(seg_pd_4_freq_slip),
              len(cross4_sms), 
              len(cross4_slip),
              len(upgrade4_sms),
              len(upgrade4_slip),
              len(ump4_sms),
              len(ump4_slip),
              len(seg_pd_4_churn_sms)+len(seg_pd_4_freq_sms) + len(cross4_sms) + len(upgrade4_sms) + len(ump4_sms),
              len(seg_pd_4_churn_slip)+len(seg_pd_4_freq_slip) + len(cross4_slip) + len(upgrade4_slip) + len(ump4_slip)))

churn_sms: 26185, churn_slip: 50406, freq_sms: 97629, freq_slip: 157423, cross_sms: 79373, cross_slip: 161093, upgrade_sms: 54238, upgrade_slip: 127842, ump_sms: 49827, ump_slip: 91174 (all_sms: 307252, all_slip: 587938)


In [38]:
seg_pd_5_churn = seg_5.join(customers_usecase_churn, 'customer_rk', 'inner').toPandas()
seg_pd_5_freq = seg_5.join(customers_usecase_freq, 'customer_rk', 'inner').toPandas()
seg_pd_5_others = seg_5.join(customers_usecase_churn, 'customer_rk', 'left_anti').join(customers_usecase_freq, 'customer_rk', 'left_anti').toPandas()

In [57]:
seg_pd_5_churn_sms = seg_pd_5_churn[seg_pd_5_churn.group == '1']['customer_rk']
seg_pd_5_churn_slip = seg_pd_5_churn[seg_pd_5_churn.group == '2']['customer_rk']
seg_pd_5_freq_sms = seg_pd_5_freq[seg_pd_5_freq.group == '1']['customer_rk']
seg_pd_5_freq_slip = seg_pd_5_freq[seg_pd_5_freq.group == '2']['customer_rk']
seg_pd_5_others_sms = seg_pd_5_others[seg_pd_5_others.group == '1']['customer_rk']
seg_pd_5_others_slip = seg_pd_5_others[seg_pd_5_others.group == '2']['customer_rk']

In [58]:
seg_pd_5_others_sms = seg_pd_5_others_sms.astype('str')
seg_pd_5_others_slip = seg_pd_5_others_slip.astype('str')

In [166]:
cross5_sms, upgrade5_sms, ump5_sms = get_usecases(pd.DataFrame({'customer_rk': seg_pd_5_others_sms.to_list()}))
cross5_slip, upgrade5_slip, ump5_slip = get_usecases(pd.DataFrame({'customer_rk': seg_pd_5_others_slip.to_list()}))

In [168]:
seg_pd_5_churn_sms = list(set(seg_pd_5_churn_sms) - set(seg_pd_4_churn_sms))
seg_pd_5_churn_slip = list(set(seg_pd_5_churn_slip) - set(seg_pd_4_churn_slip))
seg_pd_5_freq_sms = list(set(seg_pd_5_freq_sms) - set(seg_pd_4_freq_sms))
seg_pd_5_freq_slip = list(set(seg_pd_5_freq_slip) - set(seg_pd_4_freq_slip))
cross5_sms = list(set(cross5_sms['customer_rk']) - set(cross4_sms['customer_rk']))
cross5_slip = list(set(cross5_slip['customer_rk']) - set(cross4_slip['customer_rk']))
upgrade5_sms = list(set(upgrade5_sms['customer_rk']) - set(upgrade4_sms['customer_rk']))
upgrade5_slip = list(set(upgrade5_slip['customer_rk']) - set(upgrade4_slip['customer_rk']))
ump5_sms = list(set(ump5_sms['customer_rk']) - set(ump4_sms['customer_rk']))
ump5_slip = list(set(ump5_slip['customer_rk']) - set(ump4_slip['customer_rk']))

In [169]:
print('churn_sms: {}, churn_slip: {}, freq_sms: {}, freq_slip: {}, cross_sms: {}, cross_slip: {}, upgrade_sms: {}, upgrade_slip: {}, ump_sms: {}, ump_slip: {} (all_sms: {}, all_slip: {})'
      .format(len(seg_pd_5_churn_sms),
              len(seg_pd_5_churn_slip), 
              len(seg_pd_5_freq_sms),
             len(seg_pd_5_freq_slip),
              len(cross5_sms), 
              len(cross5_slip),
              len(upgrade5_sms),
              len(upgrade5_slip),
              len(ump5_sms),
              len(ump5_slip),
              len(seg_pd_5_churn_sms)+len(seg_pd_5_freq_sms) + len(cross5_sms) + len(upgrade5_sms) + len(ump5_sms),
              len(seg_pd_5_churn_slip)+len(seg_pd_5_freq_slip) + len(cross5_slip) + len(upgrade5_slip) + len(ump5_slip)))

churn_sms: 105195, churn_slip: 208630, freq_sms: 398558, freq_slip: 672215, cross_sms: 237348, cross_slip: 486028, upgrade_sms: 174692, upgrade_slip: 404815, ump_sms: 149056, ump_slip: 275858 (all_sms: 1064849, all_slip: 2047546)


In [170]:
seg_pd_6_churn = seg_6.join(customers_usecase_churn, 'customer_rk', 'inner').toPandas()
seg_pd_6_freq = seg_6.join(customers_usecase_freq, 'customer_rk', 'inner').toPandas()
seg_pd_6_others = seg_6.join(customers_usecase_churn, 'customer_rk', 'left_anti').join(customers_usecase_freq, 'customer_rk', 'left_anti').toPandas()

In [171]:
seg_pd_6_churn_sms = seg_pd_6_churn[seg_pd_6_churn.group == '1']['customer_rk']
seg_pd_6_churn_slip = seg_pd_6_churn[seg_pd_6_churn.group == '2']['customer_rk']
seg_pd_6_freq_sms = seg_pd_6_freq[seg_pd_6_freq.group == '1']['customer_rk']
seg_pd_6_freq_slip = seg_pd_6_freq[seg_pd_6_freq.group == '2']['customer_rk']
seg_pd_6_others_sms = seg_pd_6_others[seg_pd_6_others.group == '1']['customer_rk']
seg_pd_6_others_slip = seg_pd_6_others[seg_pd_6_others.group == '2']['customer_rk']

In [172]:
seg_pd_6_others_sms = seg_pd_6_others_sms.astype('str')
seg_pd_6_others_slip = seg_pd_6_others_slip.astype('str')

In [173]:
cross6_sms, upgrade6_sms, ump6_sms = get_usecases(pd.DataFrame({'customer_rk': seg_pd_6_others_sms.to_list()}))
cross6_slip, upgrade6_slip, ump6_slip = get_usecases(pd.DataFrame({'customer_rk': seg_pd_6_others_slip.to_list()}))

In [175]:
print('churn_sms: {}, churn_slip: {}, freq_sms: {}, freq_slip: {}, cross_sms: {}, cross_slip: {}, upgrade_sms: {}, upgrade_slip: {}, ump_sms: {}, ump_slip: {} (all_sms: {}, all_slip: {})'
      .format(len(seg_pd_6_churn_sms),
              len(seg_pd_6_churn_slip), 
              len(seg_pd_6_freq_sms),
             len(seg_pd_6_freq_slip),
              len(cross6_sms), 
              len(cross6_slip),
              len(upgrade6_sms),
              len(upgrade6_slip),
              len(ump6_sms),
              len(ump6_slip),
              len(seg_pd_6_churn_sms)+len(seg_pd_6_freq_sms) + len(cross6_sms) + len(upgrade6_sms) + len(ump6_sms),
              len(seg_pd_6_churn_slip)+len(seg_pd_6_freq_slip) + len(cross6_slip) + len(upgrade6_slip) + len(ump6_slip)))

churn_sms: 50421, churn_slip: 98927, freq_sms: 197443, freq_slip: 313947, cross_sms: 140604, cross_slip: 294568, upgrade_sms: 113027, upgrade_slip: 252930, ump_sms: 88890, ump_slip: 169998 (all_sms: 590385, all_slip: 1130370)


In [176]:
seg_pd_7_churn = seg_7.join(customers_usecase_churn, 'customer_rk', 'inner').toPandas()
seg_pd_7_freq = seg_7.join(customers_usecase_freq, 'customer_rk', 'inner').toPandas()
seg_pd_7_others = seg_7.join(customers_usecase_churn, 'customer_rk', 'left_anti').join(customers_usecase_freq, 'customer_rk', 'left_anti').toPandas()

In [177]:
seg_pd_7_churn_sms = seg_pd_7_churn[seg_pd_7_churn.group == '1']['customer_rk']
seg_pd_7_churn_slip = seg_pd_7_churn[seg_pd_7_churn.group == '2']['customer_rk']
seg_pd_7_freq_sms = seg_pd_7_freq[seg_pd_7_freq.group == '1']['customer_rk']
seg_pd_7_freq_slip = seg_pd_7_freq[seg_pd_7_freq.group == '2']['customer_rk']
seg_pd_7_others_sms = seg_pd_7_others[seg_pd_7_others.group == '1']['customer_rk']
seg_pd_7_others_slip = seg_pd_7_others[seg_pd_7_others.group == '2']['customer_rk']

In [178]:
seg_pd_7_others_sms = seg_pd_7_others_sms.astype('str')
seg_pd_7_others_slip = seg_pd_7_others_slip.astype('str')

In [189]:
cross7_sms, upgrade7_sms, ump7_sms = get_usecases(pd.DataFrame({'customer_rk': seg_pd_7_others_sms.to_list()}))
cross7_slip, upgrade7_slip, ump7_slip = get_usecases(pd.DataFrame({'customer_rk': seg_pd_7_others_slip.to_list()}))

In [195]:
seg_pd_7_churn_sms = list(set(seg_pd_7_churn_sms) - set(seg_pd_6_churn_sms))
seg_pd_7_churn_slip = list(set(seg_pd_7_churn_slip) - set(seg_pd_6_churn_slip))
seg_pd_7_freq_sms = list(set(seg_pd_7_freq_sms) - set(seg_pd_6_freq_sms))
seg_pd_7_freq_slip = list(set(seg_pd_7_freq_slip) - set(seg_pd_6_freq_slip))
cross7_sms = list(set(cross7_sms['customer_rk']) - set(cross6_sms['customer_rk']))
cross7_slip = list(set(cross7_slip['customer_rk']) - set(cross6_slip['customer_rk']))
upgrade7_sms = list(set(upgrade7_sms['customer_rk']) - set(upgrade6_sms['customer_rk']))
upgrade7_slip = list(set(upgrade7_slip['customer_rk']) - set(upgrade6_slip['customer_rk']))
ump7_sms = list(set(ump7_sms['customer_rk']) - set(ump6_sms['customer_rk']))
ump7_slip = list(set(ump7_slip['customer_rk']) - set(ump6_slip['customer_rk']))

In [196]:
print('churn_sms: {}, churn_slip: {}, freq_sms: {}, freq_slip: {}, cross_sms: {}, cross_slip: {}, upgrade_sms: {}, upgrade_slip: {}, ump_sms: {}, ump_slip: {} (all_sms: {}, all_slip: {})'
      .format(len(seg_pd_7_churn_sms),
              len(seg_pd_7_churn_slip), 
              len(seg_pd_7_freq_sms),
             len(seg_pd_7_freq_slip),
              len(cross7_sms), 
              len(cross7_slip),
              len(upgrade7_sms),
              len(upgrade7_slip),
              len(ump7_sms),
              len(ump7_slip),
              len(seg_pd_7_churn_sms)+len(seg_pd_7_freq_sms) + len(cross7_sms) + len(upgrade7_sms) + len(ump7_sms),
              len(seg_pd_7_churn_slip)+len(seg_pd_7_freq_slip) + len(cross7_slip) + len(upgrade7_slip) + len(ump7_slip)))

churn_sms: 214335, churn_slip: 427963, freq_sms: 852528, freq_slip: 1404411, cross_sms: 438030, cross_slip: 907443, upgrade_sms: 370366, upgrade_slip: 804817, ump_sms: 276906, ump_slip: 524769 (all_sms: 2152165, all_slip: 4069403)


In [182]:
seg_pd_8_churn = seg_8.join(customers_usecase_churn, 'customer_rk', 'inner').toPandas()
seg_pd_8_freq = seg_8.join(customers_usecase_freq, 'customer_rk', 'inner').toPandas()
seg_pd_8_others = seg_8.join(customers_usecase_churn, 'customer_rk', 'left_anti').join(customers_usecase_freq, 'customer_rk', 'left_anti').toPandas()

In [197]:
seg_pd_8_churn_sms = seg_pd_8_churn[seg_pd_8_churn.group == '1']['customer_rk']
seg_pd_8_churn_slip = seg_pd_8_churn[seg_pd_8_churn.group == '2']['customer_rk']
seg_pd_8_freq_sms = seg_pd_8_freq[seg_pd_8_freq.group == '1']['customer_rk']
seg_pd_8_freq_slip = seg_pd_8_freq[seg_pd_8_freq.group == '2']['customer_rk']
seg_pd_8_others_sms = seg_pd_8_others[seg_pd_8_others.group == '1']['customer_rk']
seg_pd_8_others_slip = seg_pd_8_others[seg_pd_8_others.group == '2']['customer_rk']

In [198]:
seg_pd_8_others_sms = seg_pd_8_others_sms.astype('str')
seg_pd_8_others_slip = seg_pd_8_others_slip.astype('str')

In [215]:
cross8_sms, upgrade8_sms, ump8_sms = get_usecases(pd.DataFrame({'customer_rk': seg_pd_8_others_sms.to_list()}))
cross8_slip, upgrade8_slip, ump8_slip = get_usecases(pd.DataFrame({'customer_rk': seg_pd_8_others_slip.to_list()}))

In [216]:
seg_pd_8_churn_sms = list(set(seg_pd_8_churn_sms) - set(seg_pd_7_churn_sms) - set(seg_pd_6_churn_sms))
seg_pd_8_churn_slip = list(set(seg_pd_8_churn_slip) - set(seg_pd_7_churn_slip) - set(seg_pd_6_churn_slip))
seg_pd_8_freq_sms = list(set(seg_pd_8_freq_sms) - set(seg_pd_7_freq_sms) - set(seg_pd_6_freq_sms))
seg_pd_8_freq_slip = list(set(seg_pd_8_freq_slip) - set(seg_pd_7_freq_slip) - set(seg_pd_6_freq_slip))
cross8_sms = list(set(cross8_sms['customer_rk']) - set(cross7_sms) - set(cross6_sms['customer_rk']))
cross8_slip = list(set(cross8_slip['customer_rk']) - set(cross7_slip) - set(cross6_slip['customer_rk']))
upgrade8_sms = list(set(upgrade8_sms['customer_rk']) - set(upgrade7_sms) - set(upgrade6_sms['customer_rk']))
upgrade8_slip = list(set(upgrade8_slip['customer_rk']) - set(upgrade7_slip) - set(upgrade6_slip['customer_rk']))
ump8_sms = list(set(ump8_sms['customer_rk']) - set(ump7_sms) - set(ump6_sms['customer_rk']))
ump8_slip = list(set(ump8_slip['customer_rk']) - set(ump7_slip) - set(ump6_slip['customer_rk']))

In [217]:
print('churn_sms: {}, churn_slip: {}, freq_sms: {}, freq_slip: {}, cross_sms: {}, cross_slip: {}, upgrade_sms: {}, upgrade_slip: {}, ump_sms: {}, ump_slip: {} (all_sms: {}, all_slip: {})'
      .format(len(seg_pd_8_churn_sms),
              len(seg_pd_8_churn_slip), 
              len(seg_pd_8_freq_sms),
             len(seg_pd_8_freq_slip),
              len(cross8_sms), 
              len(cross8_slip),
              len(upgrade8_sms),
              len(upgrade8_slip),
              len(ump8_sms),
              len(ump8_slip),
              len(seg_pd_8_churn_sms)+len(seg_pd_8_freq_sms) + len(cross8_sms) + len(upgrade8_sms) + len(ump8_sms),
              len(seg_pd_8_churn_slip)+len(seg_pd_8_freq_slip) + len(cross8_slip) + len(upgrade8_slip) + len(ump8_slip)))

churn_sms: 2029485, churn_slip: 3808769, freq_sms: 3354453, freq_slip: 5951198, cross_sms: 731468, cross_slip: 1398905, upgrade_sms: 672283, upgrade_slip: 1304247, ump_sms: 424300, ump_slip: 777999 (all_sms: 7211989, all_slip: 13241118)
